In [1]:
import numpy as np
import torch
from torch_geometric.data import Data, DataLoader
from torch_geometric.nn import GCNConv, global_mean_pool
import torch.nn.functional as F


# Define the GCN Model
class GCN(torch.nn.Module):
    """
    class GCNConv(
        in_channels: int,
        out_channels: int,
        improved: bool = False,
        cached: bool = False,
        add_self_loops: bool | None = None,
        normalize: bool = True,
        bias: bool = True,
        **kwargs: Any
    )
    """

    def __init__(self, num_features, num_outputs):
        super(GCN, self).__init__()

        self.conv1 = GCNConv(num_features, 16)
        self.conv2 = GCNConv(16, num_outputs)

    def forward(self, x, edge_index, batch):
        x = F.relu(self.conv1(x, edge_index))
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)

        # Pool node features within each graph to a single vector
        # x = global_mean_pool(x, batch)
        # return F.log_softmax(x, dim=1)
        return x


# Generate some example data
def create_random_graph_data(
    num_node_features, num_classes=None, min_nodes=5, max_nodes=20
):
    num_nodes = np.random.randint(
        min_nodes, max_nodes + 1
    )  # Randomly choose the number of nodes
    edge_prob = 0.2  # Probability of edge creation between any two nodes
    edge_index = []
    for i in range(num_nodes):
        for j in range(num_nodes):
            if i != j and np.random.rand() < edge_prob:
                edge_index.append([i, j])
    edge_index = (
        torch.tensor(edge_index).t().contiguous()
    )  # Transpose and make contiguous
    x = torch.randn(num_nodes, num_node_features)  # Node features
    # y = torch.randint(0, num_classes, (1,)).item()  # Graph-level label
    # return Data(x=x, edge_index=edge_index, y=y)
    return Data(x=x, edge_index=edge_index)


num_graphs = 4

# Use the new function to create a dataset with more varied graph sizes
graphs = [create_random_graph_data(3, 2) for _ in range(num_graphs)]

# DataLoader for batching graphs
loader = DataLoader(graphs, batch_size=4, shuffle=False)

# Initialize the model
model = GCN(num_features=3, num_outputs=8)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# Train the model
model.train()
for epoch in range(10):
    for batch in loader:
        optimizer.zero_grad()
        out = model(batch.x, batch.edge_index, batch.batch)

        print(out.shape)

        loss = F.nll_loss(out, batch.y)
        loss.backward()
        optimizer.step()
        print(f"Epoch {epoch}, Loss: {loss.item()}")

/home/tk/.virtualenvs/agent-room-env-v2-gnn/lib/python3.10/site-packages/torch_geometric/deprecation.py:22: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


torch.Size([37, 8])


TypeError: nll_loss_nd(): argument 'target' (position 2) must be Tensor, not NoneType

In [2]:
graphs

[Data(x=[7, 3], edge_index=[2, 10]),
 Data(x=[18, 3], edge_index=[2, 62]),
 Data(x=[5, 3], edge_index=[2, 5]),
 Data(x=[7, 3], edge_index=[2, 14])]

In [3]:
graphs[-1].edge_index

tensor([[0, 0, 1, 1, 1, 2, 2, 3, 4, 4, 4, 4, 6, 6],
        [4, 6, 0, 2, 4, 0, 6, 0, 0, 1, 2, 6, 0, 5]])

In [4]:
batch

DataBatch(x=[37, 3], edge_index=[2, 91], batch=[37], ptr=[5])

In [5]:
batch.x

tensor([[-1.8557,  1.6845, -0.5340],
        [-0.3092, -1.7935, -1.5452],
        [ 0.3552,  0.0205, -0.5432],
        [-0.3292,  0.1938,  0.0675],
        [ 1.5706,  0.8971,  0.3783],
        [-0.7358, -1.5291,  1.9143],
        [-0.4175,  1.9325,  0.4126],
        [-0.3053,  0.1060,  1.8944],
        [-0.0402, -0.1196,  1.7489],
        [ 0.5083, -0.6034, -1.2236],
        [-1.0666, -0.8047, -0.5614],
        [ 1.0300, -0.5257, -0.5523],
        [ 0.2052,  0.0303,  1.3498],
        [ 0.7831,  0.4623,  0.8327],
        [-1.0730, -0.0708,  0.5925],
        [-0.8542,  0.4183, -0.6244],
        [-0.3648, -0.3940,  0.8341],
        [-0.5207,  0.0608, -0.5262],
        [ 0.3114,  1.6071, -0.3523],
        [ 1.2595,  0.3296, -0.7364],
        [ 0.2573, -0.1825,  0.9415],
        [-3.1771, -1.8843,  0.6215],
        [ 0.8237, -0.9621,  0.6471],
        [ 1.0536, -0.5551,  0.3413],
        [-0.4691, -1.2372,  0.2933],
        [ 0.0110, -1.7186,  2.4158],
        [-0.8785, -0.6887, -0.7834],
 

In [6]:
batch.edge_index

tensor([[ 1,  2,  2,  3,  4,  4,  4,  5,  5,  6,  7,  7,  8,  8,  8,  8,  9,  9,
          9,  9,  9, 10, 10, 10, 10, 11, 11, 11, 12, 12, 12, 12, 12, 13, 14, 14,
         14, 14, 14, 14, 15, 15, 15, 15, 16, 17, 17, 17, 18, 18, 18, 18, 18, 18,
         19, 19, 19, 19, 19, 19, 20, 20, 20, 21, 21, 21, 21, 22, 23, 24, 24, 24,
         25, 26, 26, 28, 28, 30, 30, 31, 31, 31, 32, 32, 33, 34, 34, 34, 34, 36,
         36],
        [ 2,  1,  5,  6,  2,  5,  6,  2,  3,  3, 13, 20, 14, 16, 17, 19, 10, 13,
         21, 22, 23,  7, 11, 15, 23,  7, 12, 18, 13, 14, 16, 17, 18, 17,  7,  9,
         10, 18, 21, 22,  9, 10, 17, 18, 22, 10, 18, 19,  9, 15, 16, 20, 23, 24,
          9, 11, 13, 16, 23, 24,  7, 13, 17, 13, 15, 17, 23, 11, 11,  9, 10, 17,
         26, 25, 29, 25, 26, 34, 36, 30, 32, 34, 30, 36, 30, 30, 31, 32, 36, 30,
         35]])

In [7]:
batch.batch

tensor([0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3])